# Radio Transcrtiption using Whisper

This notebook will implement the necessary code for making the transcription of all our team message radios.


First, I need to **import the modules** and **verifying cuda is detecting my GPU**:

---

In [3]:
import os
import glob
import pandas as pd
import torch
import whisper
from tqdm.notebook import tqdm
import librosa
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

CUDA available: True
CUDA device: NVIDIA GeForce RTX 4060 Laptop GPU


## Loading audio files

---

In [5]:
def load_audio_files(audio_dir = "../../f1-strategy/data/audio"):
    """
    Load all audio files from the directory structure.
    
    Parameters:
    -----------
    audio_dir : str
        Path to the main audio directory that contains driver subdirectories
        
    Returns:
    --------
    list
        List of dictionaries containing audio file paths and metadata
    """
    audio_files = []


    # Find all driver directories. For this, we use driver_* for matching every directory

    driver_dirs = glob.glob(os.path.join(audio_dir, 'driver_*'))

    for driver_dir in driver_dirs:
        # Extract the driver number out of the directory name

        driver_num = os.path.basename(driver_dir).replace('driver_(', '').replace(',)', '')
        # Find all audio files in this driver directory.
        # For now, only mp3 is in our directory. 
        # However, we add all those extensions if the directory changes in the future
        files = glob.glob(os.path.join(driver_dir, '*.mp3')) + \
                glob.glob(os.path.join(driver_dir, '*.wav')) + \
                glob.glob(os.path.join(driver_dir, '*.m4a')) + \
                glob.glob(os.path.join(driver_dir, '*.ogg'))
        
        for file_path in files:
            # Get the filename without the extension
            filename = os.path.basename(file_path)

            # Add to the list of audio files
            audio_files.append(
                {
                    "driver": driver_num,
                    "file_path": file_path,
                    "filename": filename
                }
            )
    print(f"Found {len(audio_files)} audio files across {len(driver_dir)} driver directories")

    return audio_files

Now, I´ll store all this audio files in a variable for transcribing them with Whisper

In [6]:
# EXECUTION CELL

audio_files = load_audio_files()

# Display the first few files with a simple verifying
if audio_files:
    print("\nFirst 5 audio files")
    for file in audio_files[:5]:
        print(f"Driver: {file['driver']}, File: {file['filename']}")
else:
    print("No audio files found. Check the path or add more debugging messages")

Found 210 audio files across 41 driver directories

First 5 audio files
Driver: 1, File: driver_(1,)_belgium_radio_39.mp3
Driver: 1, File: driver_(1,)_belgium_radio_40.mp3
Driver: 1, File: driver_(1,)_belgium_radio_60.mp3
Driver: 1, File: driver_(1,)_belgium_radio_62.mp3
Driver: 1, File: driver_(1,)_belgium_radio_63.mp3


## Transcribring the audios with Whisper